In [ ]:
import pandas as pd
from tqdm import tqdm
import pickle
import resume_preprocessing_util
from Skill_extractor import Skill_extractor
from education_extractor import Education_extractor

In [ ]:
resume_df = pd.read_csv('dataset/resume_raw_data.csv', index_col=0)

In [ ]:
resume_df

In [ ]:
resume_df['resume'] = resume_df['resume'].apply(lambda x: x.lower())
resume_df['tokens'] = resume_df['resume'].apply(lambda x: resume_preprocessing_util.tokenize(x))

flag_list = ['education', 'summary', 'experience']
flag_dict = {'education' : [['educational', 'background'],
                            'education', 'educational', 'academic'], 
             'summary' : ['summary'], 
             'experience' : [['professional', 'experience'], 
                             ['work', 'experience'], 
                             ['work', 'history'],
                             'experience']}
for flag in flag_list:
    resume_df[f'{flag}_start_index'] = resume_df['tokens'].apply(lambda x: resume_preprocessing_util.find_index_in_tokens(x, flag_dict[flag]))
for flag in flag_list:
    resume_df[f'{flag}_end_index'] = resume_df.apply(lambda x: resume_preprocessing_util.find_end_index(x, flag, flag_list), axis=1)
for flag in flag_list:
    resume_df[f'{flag}_tokens'] = resume_df.apply(lambda x: resume_preprocessing_util.split_part(x, flag), axis=1)
    
resume_df['education_section'] = resume_df['education_tokens'].apply(lambda x: ' '.join(x))
resume_df['experience_section'] = resume_df['experience_tokens'].apply(lambda x: ' '.join(x))
resume_df['year_experience'] = resume_df.apply(lambda x: resume_preprocessing_util.find_year_experience(x['tokens'], x['experience_start_index'], x['experience_end_index']), axis=1)

resume_df

In [ ]:
skill_extractor = Skill_extractor()
for i in tqdm(range(len(resume_df['experience_section']))):
    result =  skill_extractor.skill_extract(resume_df['experience_section'][i])
    with open(f'resume_skills/{i}.pickle', 'wb') as f:
        pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
all_skill_dict = []
for i in tqdm(range(len(resume_df['experience_section']))):
    with open(f'resume_skills/{i}.pickle', 'rb') as f:
        temp = pickle.load(f)
        all_skill_dict.append(temp)
resume_df['all_skill_dict'] = all_skill_dict
resume_df['hard_skills'] = resume_df['all_skill_dict'].apply(lambda x: x['Hard Skill'] if 'Hard Skill' in x else [])
resume_df['soft_skills'] = resume_df['all_skill_dict'].apply(lambda x: x['Soft Skill'] if 'Soft Skill' in x else [])

In [ ]:
education_extractor = Education_extractor()
for i in tqdm(range(len(resume_df['education_section']))):
    result =  education_extractor.extract_education_features(resume_df['education_section'][i])
    with open(f'resume_edu/{i}.pickle', 'wb') as f:
        pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
all_edu_dict = []
for i in tqdm(range(len(resume_df['education_section']))):
    with open(f'resume_edu/{i}.pickle', 'rb') as f:
        temp = pickle.load(f)
        all_edu_dict.append(temp)
resume_df['all_edu_dict'] = all_edu_dict
resume_df['majors'] = resume_df['all_edu_dict'].apply(lambda x: x['majors'] if 'majors' in x else [])
resume_df['degrees'] = resume_df['all_edu_dict'].apply(lambda x: x['degrees'] if 'degrees' in x else [])
resume_df['org'] = resume_df['all_edu_dict'].apply(lambda x: x['org'] if 'org' in x else [])
resume_df['org_rank'] = resume_df['all_edu_dict'].apply(lambda x: x['org_rank'] if 'org_rank' in x else -1)

In [ ]:
resume_df

In [ ]:
# resume_df.to_csv("dataset/resume_final_data.csv")